In [ ]:
### RHLF - LLM Model


from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

# Sample patient question-answer pairs
patient_data = [
    {"question": "What are the symptoms of COVID-19?", "answer": "Common symptoms of COVID-19 include fever, cough, and shortness of breath."},
    {"question": "How is diabetes diagnosed?", "answer": "Diabetes is diagnosed through tests such as fasting blood sugar, oral glucose tolerance test, etc."},
    {"question": "What treatments are available for asthma?", "answer": "Treatments for asthma include bronchodilators, inhaled corticosteroids, etc."},
    # Add more patient questions and corresponding answers
]

# Load pre-trained Sentence Transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
df = pd.DataFrame(patient_data)

# Embed the answers


def calculate_similarity(query, embeddings):
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    return cos_scores

def input_text_model(input_text):
    input_sentence_processed = ''.join([i for i in input_text if not i.isdigit()])
    query = input_sentence_processed
    answer_embeddings = model.encode(df['question'], convert_to_tensor=True)
    query_similarity = calculate_similarity(query, answer_embeddings)
    top_n = 1
    top_indices = query_similarity.argsort().cpu().numpy()[-top_n:]
    similar_answers = df.loc[top_indices, 'answer'].tolist()
    return similar_answers[-1]

# Collect human feedback and update model
def update_model_with_feedback(input_text, human_feedback):
    # Incorporate human feedback into training data
    df.loc[len(df)] = {'question': input_text, 'answer': human_feedback}
    answer_embeddings = model.encode(df['question'], convert_to_tensor=True)
    input_sentence_processed = ''.join([i for i in input_text if not i.isdigit()])
    query = input_sentence_processed
    query_similarity = calculate_similarity(query, answer_embeddings)
    top_n = 1
    top_indices = query_similarity.argsort().cpu().numpy()[-top_n:]
    similar_answers = df.loc[top_indices, 'answer'].tolist()
    return similar_answers[-1]
    # Re-embed updated answers
    #answer_embeddings = model.encode(df['answer'], convert_to_tensor=True)

# Example usage with feedback loop
patient_question = "symptoms of diabetics?"
initial_answer = input_text_model(patient_question)
print("Initial Answer:", initial_answer)

# Assume human provides feedback
yes_no=input("Are you happy with the answer?")
if yes_no!="Yes":
    human_feedback =input("Provide me the data to train")
    #human_feedback = "The answer provided was helpful, but it could include more details about preventive measures."
     
    updated_answer = update_model_with_feedback(patient_question, human_feedback)
    print("Updated Answer:", updated_answer)
else :
    print("Happy for guiding you") 